<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [10]:
# before everything, import these libraries (Shift-Enter)

import sys, os
import numpy as np
import xarray as xr
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as ipw
from IPython.lib.display import YouTubeVideo
from numba import njit, jit

sys.path.append(os.path.abspath('../../src/Ipywidgets-Scripts/'))
from scripts.widgets_viz import SliderMatrix

# II. Coding Projects > TITLE


<hr>
&nbsp;

In [2]:
YouTubeVideo('',width=560,height=315)

In [3]:
def initialize(x0, y0):
    global x, y, xresult, yresult
    x = x0
    y = y0
    xresult = [x]
    yresult = [y]


def observe():
    global x, y, xresult, yresult
    xresult.append(x)
    yresult.append(y)


def update(par):
    global x, y, xresult, yresult
    a,b,c,d = par.squeeze()
#     nextx = a.val*x + b.val*y
#     nexty = -a.val*x + b.val*y
    nextx = a*x + b*y
    nexty = -a*x + b*y
    x, y = nextx, nexty


def display(n):
    global xresult, yresult
    fig1 = go.FigureWidget(make_subplots(rows=2,  subplot_titles=("Values", "Phase Space")))
    fig1.add_trace(go.Scatter(x=np.arange(n), y=xresult, name='x'), row=1, col=1)
    fig1.add_trace(go.Scatter(x=np.arange(n), y=yresult, name='y'), row=1, col=1)
    fig1.add_trace(go.Scatter(x=xresult, y=yresult, showlegend=False), row=2, col=1)
    fig1.update_layout(template='none', width=800, height=500)
    fig1.update_xaxes(title_text="x", row=2, col=1)
    fig1.update_yaxes(title_text="y", row=2, col=1)
#     return ipw.VBox([ipw.HBox([a.slider, b.slider]), fig1])
    return fig1

def main(ini, par, n):
    x0, y0 = ini
    global x, y, xresult, yresult
    initialize(x0, y0)
    for _ in range(n):
        update(par)
        observe()
    return display(n)
    # return xresult

In [4]:
main(ini=(1,1), par=(0.5,2,0,0), n=15)

FigureWidget({
    'data': [{'name': 'x',
              'type': 'scatter',
              'uid': '0c9be25a-3e9e…

In [32]:
par = SliderMatrix("par", [[(0,4,1), (0,4,1)], [(0,4,1), (0,4,1)]])
par.show()

GridspecLayout(children=(FloatSlider(value=0.0, description='par(0,0)', layout=Layout(grid_area='widget001'), …

In [101]:
v1 = np.arange(0,10,1)
v2 = np.arange(10,100,25)
v3 = np.arange(100,1000,500)
vectors = (v1, v2, v3)

In [3]:
v1 = np.arange(1,2,1)
v2 = np.arange(10,40,10)
v3 = np.arange(100,600,100)
v4 = np.arange(1000,8000,1000)
# v5 = np.arange(10000,50000,10000)
# v6 = np.arange(100000,500000,100000)
# vectors = (v1, v2, v3, v4, v5, v6)
vectors = (v1, v2, v3, v4)

shape = (2,2)
vec_space = vector_space(vectors, shape=shape)
vec_space.shape

# res = xr.DataArray(vector_space(vectors), 
#                     coords=vectors, 
#                     dims=['v1', 'v2' ,'v3', 'v4']
#                     )
# res

(1, 3, 5, 7, 2, 2)

In [6]:
from functools import partial

In [11]:
@partial(jit, static_argnums=(0,))
def trace(func, matrix_list):
    dim = matrix_list.shape[0]
    trace = np.zeros(shape=dim, dtype=np.float32)
    for i in range(dim):
        trace[i] = func(matrix_list[i])
        trace[i] = func(matrix_list[i])
    return trace


def test(vec_space, shape):
    prev_type = vec_space.dtype
    dim_0 = vec_space.shape[:-len(shape)]
    flat_arr = vec_space.reshape(np.prod(dim_0), *shape).astype(np.float32)
    res = trace(np.trace, flat_arr)
    return res.reshape(dim_0).astype(prev_type, casting='unsafe')


def test2(vec_space, shape):
    prev_type = vec_space.dtype
    dim_0 = vec_space.shape[:-len(shape)]
    flat_arr = vec_space.reshape(np.prod(dim_0), *shape).astype(np.float32)
    res = np.trace(flat_arr, axis1=1, axis2=2)
    return res.reshape(dim_0).astype(prev_type, casting='unsafe')

def test3(func, vec_space, shape):
    prev_type = vec_space.dtype
    dim_0 = vec_space.shape[:-len(shape)]
    flat_arr = vec_space.reshape(np.prod(dim_0), *shape).astype(np.float32)
    func = njit()(func)
    res = trace(np.trace, flat_arr)
    return res.reshape(dim_0).astype(prev_type, casting='unsafe')


In [8]:
%timeit test(vec_space, shape)

KeyError: "Unrecognized options: {'static_argnums'}. Known options are dict_keys(['_nrt', 'boundscheck', 'debug', 'error_model', 'fastmath', 'forceobj', 'inline', 'looplift', 'no_cfunc_wrapper', 'no_cpython_wrapper', 'no_rewrites', 'nogil', 'nopython', 'parallel', 'target_backend'])"

In [240]:
%timeit test2(vec_space, shape)

16.6 µs ± 1.69 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [245]:
func = {'trace': np.trace}
func['trace'](vec_space[0,0,0,0])

1001

In [214]:
def revert_cast(arr):
    prev_type = arr.dtype
    new_arr = arr.astype(np.float32)
    return new_arr.astype(prev_type, casting='equiv')

revert_cast(np.array([0,1,2,3,4,5]))

TypeError: Cannot cast array data from dtype('float32') to dtype('int64') according to the rule 'equiv'

In [221]:
%%timeit
shape = (2,2)
# new_arr = vec_space.reshape(-1, *shape).astype('float32')
temp = test(vec_space, shape)
# dim = vec_space.shape[:-len(shape)]
# np.prod(dim)
temp

The slowest run took 4.66 times longer than the fastest. This could mean that an intermediate result is being cached.
31.6 µs ± 22.6 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [173]:
np.array(vec_space.shape)[:-len(shape)]

array([1, 3, 5, 7])

In [59]:
temp

array([1001., 2001., 3001., 4001., 5001., 6001., 7001., 1001., 2001.,
       3001., 4001., 5001., 6001., 7001., 1001., 2001., 3001., 4001.,
       5001., 6001., 7001., 1001., 2001., 3001., 4001., 5001., 6001.,
       7001., 1001., 2001., 3001., 4001., 5001., 6001., 7001., 1001.,
       2001., 3001., 4001., 5001., 6001., 7001., 1001., 2001., 3001.,
       4001., 5001., 6001., 7001., 1001., 2001., 3001., 4001., 5001.,
       6001., 7001., 1001., 2001., 3001., 4001., 5001., 6001., 7001.,
       1001., 2001., 3001., 4001., 5001., 6001., 7001., 1001., 2001.,
       3001., 4001., 5001., 6001., 7001., 1001., 2001., 3001., 4001.,
       5001., 6001., 7001., 1001., 2001., 3001., 4001., 5001., 6001.,
       7001., 1001., 2001., 3001., 4001., 5001., 6001., 7001., 1001.,
       2001., 3001., 4001., 5001., 6001., 7001.], dtype=float32)

In [40]:
new_arr = vec_space.reshape(-1,*shape)
new_arr.shape

(105, 2, 2)

In [55]:
@njit
def test(arr, dim):
    res = np.zeros()
    return np.array([np.trace(arr[i]) for i in range(arr.shape[dim])])

In [56]:
# %%timeit
# np.array([np.trace(new_arr[i]) for i in range(new_arr.shape[0])])
test(new_arr, 0)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1mNo implementation of function Function(<function trace at 0x7f1ea8093dc0>) found for signature:
 
 >>> trace(array(int64, 2d, A))
 
There are 2 candidate implementations:
[1m  - Of which 2 did not match due to:
  Overload in function 'matrix_trace_impl': File: numba/np/linalg.py: Line 2566.
    With argument(s): '(array(int64, 2d, A))':[0m
[1m   Rejected as the implementation raised a specific error:
     TypingError: np.trace() only supported on float and complex arrays.[0m
  raised from /home/adrien/Documents/Apps/Miniconda/miniconda3/envs/minimal-env/lib/python3.9/site-packages/numba/np/linalg.py:773
[0m
[0m[1mDuring: resolving callee type: Function(<function trace at 0x7f1ea8093dc0>)[0m
[0m[1mDuring: typing of call at /tmp/ipykernel_189716/3663710354.py (3)
[0m
[1m
File "../../../../../../../../tmp/ipykernel_189716/3663710354.py", line 3:[0m
[1m<source missing, REPL/exec in use?>[0m


In [51]:
def f(x):
    return x*x

def call_func(func, x):
    func = njit(func)   # compile func in nopython mode!
    @njit
    def inner(x):
        return func(x)
    return inner(x)

In [54]:
%timeit call_func(f,5)

78.4 ms ± 8.44 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [169]:
%%timeit
np.trace(new_arr, axis1=1, axis2=2)

4.04 µs ± 269 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [ ]:
np.linalg.det(

In [ ]:
%%timeit
np.array([np.trace(new_arr[i]) for i in range(new_arr.shape[0])])

In [167]:
np.apply_along_axis(np.trace, 0, new_arr)

ValueError: diag requires an array of at least two dimensions

In [124]:
np.trace(vec_space2.reshape(2,2,-1)[:,:,0])

1000

In [144]:
np.trace(arr[2])

1000

In [143]:
arr = vec_space.reshape(-1,2,2)
np.array([np.trace(mat for mat in arr)])

ValueError: diag requires an array of at least two dimensions

In [ ]:
np.linalg.det(vec_space.reshape(2,2,-1))

In [105]:
vec_space2 = vector_space2(vectors, shape=(2,2))
mytrace = np.linalg.det(vec_space2[:,:,:,:])
mytrace.shape
mytrace

array([[[[0., 0.],
         [0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.],
         [0., 0.]]]])

In [106]:
def apply_to_slices(func, a, axes):
    """Apply 'func' to each slice of array 'a', where a slice spans 'axes'.

    Args:
    func: function expecting an array of rank len(axes) and returning a
        modified array of the same shape.
    a: input array of arbitrary shape.
    axes: integer sequence specifying the slice orientation.
    """
    # The approach is to move the slice axes to the end of the array, reshape to
    # a 1-D array of slices, apply the user function to each slice, reshape back
    # to an outer array of slices, and finally move the slice axes back to their
    # original locations.  https://stackoverflow.com/a/61297133/
    assert len(axes) <= a.ndim
    outer_ndim = a.ndim - len(axes)
    a = np.moveaxis(a, axes, range(outer_ndim, a.ndim))
    outer_shape = a.shape[:outer_ndim]
    slice_shape = a.shape[outer_ndim:]
    a = a.reshape((-1,) + slice_shape)
    a = np.array([func(a_slice) for a_slice in a])
    a = a.reshape(outer_shape + slice_shape)
    a = np.moveaxis(a, range(outer_ndim, a.ndim), axes)
    return a

In [104]:
np.all(res==mytrace)

/tmp/ipykernel_145327/4156889803.py:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  np.all(res==mytrace)


False

In [71]:
np.linalg.det(vec_space[0,0,0,0])

-1000.0000000000007

In [73]:
np.trace(vec_space[:,:,:,:]).shape

(4, 4, 2, 2)

In [72]:

res = xr.DataArray(np.trace(vec_space[:,:,:,:]), 
                    # coords=(v4, v2, v3, v1), 
                    # dims=['v1', 'v2', 'v3', 'v4']
                    )
res.shape

(4, 4, 2, 2)

In [30]:
res = vector_space(vectors, shape=(2,2))
res[0,0,0,0]

array([[  0,  10],
       [-10,  -5]])

In [40]:
res.shape

(5, 5, 5, 5, 2, 2)

In [32]:
np.linalg.det(res[0,0,0,0])

100.00000000000004

In [42]:
np.linalg.det(res[:,:,:,:])[0,0,0,0]

100.00000000000004

In [36]:
m = [[253.0, 32581341.0, 16387064.0], 
     [253.0, 32581088.0, 16387064.0], 
     [253.0, 16322548.0, 16387064.0]]
round(np.linalg.det(m))

8

In [12]:
from scipy.integrate import odeint

In [27]:
@njit
def model_njit(y, t, decay):
    return -decay * y

def model_gen(decay):
    def inner(y, t):
        return model_njit(y, t, decay)
    return inner

# parameters = np.arange(10)/100. # spoof parameters

# for i in parameters:
#     model = model_gen(i)
#     y0 = 1
#     t = np.arange(10)
#     y = odeint(model, y0, t)
#     plt.plot(t, y)

# print(model_njit.signatures)
# plt.show()